# Random Feature Generator
---

### Hyper-parameter(s)

In [240]:
# Desired Number of Parts (Number of Feature Maps - 1 (for identity map))
N_parts = 5

# How many feature maps should be generated?
N_Features_Search_Space_Dimension = 200

#### Import Packages

In [241]:
# Import from Package-dump
exec(open('init.py').read())

#### Read Data

In [227]:
# load dataset
data_path_folder = "/scratch/users/kratsioa/Desktop/Architopes-master_swish_random_partition/data/data/"

# Read Data
data_path = data_path_folder+"housing_complete.csv"
X = pd.read_csv(data_path)

In [228]:
# Load Inputs
X_train = X[X['is_train']==True]
X_test = X[X['is_train']==False]
# Load Outputs
y_train = X_train['median_house_value']
y_test = X_test['median_house_value']
# Remove Columns Label Columns
del X_train['is_train']
del X_test['is_train']
del X_train['median_house_value']
del X_test['median_house_value']
# Remember Column Names
X_colnames = X_train.columns

---

### Helper Function(s):

In [229]:
# Univariate Compositer
def compositer_univariate(x):
    # Apply Activation
    x_out = 1 / (1 + np.math.exp(-x)) 
    # Output
    return x_out

# Vectorize
compositer = np.vectorize(compositer_univariate)

def activ_univariate(x):
    if x<0:
        x_out = 0.9*np.sign(x)*np.math.pow(-x,2)
    else:
        x_out = 0.1*np.math.pow(x,2)
    return x_out

# Vectorize
activ = np.vectorize(activ_univariate)

def activ_univariate_inv(x):
    if x<0:
        x_out = (1/0.9)*np.sign(x)*np.sqrt(-x)
    else:
        x_out = (1/0.1)*np.sqrt(x)
    return x_out

# Vectorize
compositer = np.vectorize(activ_univariate_inv)

#### Implied Hyper-parameter(s)

In [230]:
# Remove 1 part because of the identity map!
N_parts = max(1,N_parts)

# Ensure that there are at-least as many random feature maps generated as there are parts to the partition!
N_Features_Search_Space_Dimension = max(N_parts,N_Features_Search_Space_Dimension)

---

### Generate Random Features

#### Build Base "Shallow" Prediction (Identity Map)

In [231]:
# Initialize Regressor
Loop_Regressor = LinearRegression()
Loop_Regressor.fit(X_train,y_train)

# Fit Regressor
feature_predictions = np.array(np.abs(Loop_Regressor.predict(X_train)-y_train))
feature_predictions = feature_predictions.reshape(feature_predictions.shape[0],1)

#### Initialize Parameters

In [232]:
### Initialize Parameters
#------------------------#
# Initialize History
past_val = -1
current_position = 0
# Initalize Features
X_train_features = X_train
X_test_features = X_test

# Construct Deep Randomized Features
#------------------------------------#
# Set Seed
np.random.seed(2020)

### Builds "Deep" Features
*Note:* We always force the identity to be a potentially valid representation.  Therefore, it is not score.

In [233]:
if N_Features_Search_Space_Dimension>0:
    for i in range(N_Features_Search_Space_Dimension):    
        # Transformations
        #-----------------#
        # Generate Random Weights and Biases
        Weights_random = (np.random.binomial(1,.5,(X_train_features.shape[1],X_train_features.shape[1])) - .5)*2
        biases_random = (np.random.binomial(1,.5,X_train_features.shape[1]) -.5)*2         
        
        # Write Non-Liearly Transformed Features only if transformation has been applied, only if Depth >0
        # Apply Activation
        X_train_features = compositer(X_train_features)
        X_test_features = compositer(X_test_features)
        # Apply Random Weights
        X_train_features = np.matmul(X_train_features,Weights_random)
        X_test_features = np.matmul(X_test_features,Weights_random)
        # # Apply Bias        
        X_train_features = X_train_features + biases_random
        X_test_features = X_test_features + biases_random


        # Register Best Scores
        #----------------------#
        # Make Predictions 
        Loop_Regressor = LinearRegression()
        Loop_Regressor.fit(X_train,y_train)
        # Evaluate Errors
        loop_prediction_errors = np.array(np.abs(Loop_Regressor.predict(X_train_features)-y_train))
        loop_prediction_errors = loop_prediction_errors.reshape(loop_prediction_errors.shape[0],1)
        feature_predictions = np.append(feature_predictions,loop_prediction_errors,axis=1)
        # Register Score
        score_loop = np.mean(feature_predictions)
        if(i == 0):
            # Initialize Score
            score = score_loop
        else:
            # Update Score
            score = np.append(score,score_loop)
        
        # Update User #
        #-------------#
        print('FG-Progress:' + str(i/N_Features_Search_Space_Dimension))

# Update User
#-------------#
print('Feature Generation (Learning Phase): Score Generated')

# Determined Best Features
#--------------------------#
best_features = np.argsort(score)[0:N_parts]
print(best_features)

# Generate Partition Training Tarets #
#------------------------------------#
# Extract Only the Relevant Best Predictions
partition_labels_training = feature_predictions[:,best_features]
partition_labels_training = np.argmin(partition_labels_training, axis=1)

# Reset Seed
np.random.seed(2020)

FG-Progress:0.0
FG-Progress:0.005
FG-Progress:0.01
FG-Progress:0.015
FG-Progress:0.02
FG-Progress:0.025
FG-Progress:0.03
FG-Progress:0.035
FG-Progress:0.04
FG-Progress:0.045
FG-Progress:0.05
FG-Progress:0.055
FG-Progress:0.06
FG-Progress:0.065
FG-Progress:0.07
FG-Progress:0.075
FG-Progress:0.08
FG-Progress:0.085
FG-Progress:0.09
FG-Progress:0.095
FG-Progress:0.1
FG-Progress:0.105
FG-Progress:0.11
FG-Progress:0.115
FG-Progress:0.12
FG-Progress:0.125
FG-Progress:0.13
FG-Progress:0.135
FG-Progress:0.14
FG-Progress:0.145
FG-Progress:0.15
FG-Progress:0.155
FG-Progress:0.16
FG-Progress:0.165
FG-Progress:0.17
FG-Progress:0.175
FG-Progress:0.18
FG-Progress:0.185
FG-Progress:0.19
FG-Progress:0.195
FG-Progress:0.2
FG-Progress:0.205
FG-Progress:0.21
FG-Progress:0.215
FG-Progress:0.22
FG-Progress:0.225
FG-Progress:0.23
FG-Progress:0.235
FG-Progress:0.24
FG-Progress:0.245
FG-Progress:0.25
FG-Progress:0.255
FG-Progress:0.26
FG-Progress:0.265
FG-Progress:0.27
FG-Progress:0.275
FG-Progress:0.28
FG-Pro

In [234]:
# Update User
print("The best " + str((N_parts + 1)) + " features, indexed in order, are: " + str(best_features) + "... Out of: " + str(N_Features_Search_Space_Dimension)+ " generated in total!")

The best 6 features, indexed in order, are: [3 6 2 5 4]... Out of: 200 generated in total!


---

### Train Classifier

---

In [ ]:
if N_Features_Search_Space_Dimension>0:
    for i in range(min(N_Features_Search_Space_Dimension,max(best_features))): # don't run everything if maximum is attained before iteration limit!
        # Transformations
        #-----------------#
        # Generate Random Weights and Biases
        Weights_random = (np.random.binomial(1,.5,(X_train_features.shape[1],X_train_features.shape[1])) - .5)*2
        biases_random = (np.random.binomial(1,.5,X_train_features.shape[1]) -.5)*2         
        
        # Write Non-Liearly Transformed Features only if transformation has been applied, only if Depth >0
        # Apply Activation
        X_train_features = compositer(X_train_features)
        X_test_features = compositer(X_test_features)
        # Apply Random Weights
        X_train_features = np.matmul(X_train_features,Weights_random)
        X_test_features = np.matmul(X_test_features,Weights_random)
        # # Apply Bias        
        X_train_features = X_train_features + biases_random
        X_test_features = X_test_features + biases_random


        # Register Best Scores
        #----------------------#
        Loop_Regressor = LinearRegression()
        Loop_Regressor.fit(X_train,y_train)
        if i in best_features:
            # Generate Array of Features to Write
            X_Features_to_write_train = pd.DataFrame(X_train_features,columns=X_colnames).join(y_train).assign(is_train=True)
            X_Features_to_write_test = pd.DataFrame(X_test_features,columns=X_colnames).join(y_test).assign(is_train=False)
            X_Features_to_write = X_Features_to_write_train.append(X_Features_to_write_test, ignore_index=True)
            # Write Features
            
        # Update User #
        #-------------#
        print('FG-Progress:' + str(i/N_Features_Search_Space_Dimension))

# Update User
#-------------#
print('Feature Generation (Learning Phase): Score Generated')

# Determined Best Features
#--------------------------#
best_features = np.argsort(score)[0:N_parts]
print(best_features)